In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from huggingface_hub import notebook_login

In [3]:
notebook_login()

In [4]:
df = pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df['Category'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
df.head()

,review,sentiment,Category
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [6]:
df.shape

(50000, 3)

In [7]:
df = df.rename(columns={
    'review': 'text',
    'Category': 'label'
})


In [8]:
df.label.value_counts()

label
1    25000
0    25000
Name: count, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test = train_test_split(df, test_size = 0.2)
X_train, X_val = train_test_split(X_train, test_size = 0.2) 

In [11]:
print(X_train.head())

                                                    text sentiment  label
2985   I'm sure this is a show no one is that familia...  positive      1
48895  Dudley Moore is fantastic in this largley unkn...  positive      1
42343  ...instead, watch it as a great coming of age ...  positive      1
34998  Aardman does it again. Next to Pixar, Aardman ...  positive      1
34841  Interesting idea and storyline which didn't qu...  negative      0


In [12]:
from datasets import Dataset

In [13]:
train_ds = Dataset.from_pandas(X_train, split="train")
val_ds = Dataset.from_pandas(X_val, split = 'test')
test_ds = Dataset.from_pandas(X_test, split="test")

In [14]:
train_ds

Dataset({
    features: ['text', 'sentiment', 'label', '__index_level_0__'],
    num_rows: 32000
})

In [15]:
val_ds

Dataset({
    features: ['text', 'sentiment', 'label', '__index_level_0__'],
    num_rows: 8000
})

In [16]:
test_ds

Dataset({
    features: ['text', 'sentiment', 'label', '__index_level_0__'],
    num_rows: 10000
})

In [17]:
from transformers import AutoTokenizer

In [18]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [19]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [20]:
tokenized_train = train_ds.map(preprocess_function, batched=True)
tokenized_val = val_ds.map(preprocess_function, batched=True)
tokenized_test = test_ds.map(preprocess_function, batched=True)


Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [21]:
from transformers import DataCollatorWithPadding

In [22]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [23]:
import evaluate
import numpy as np

In [24]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [25]:
id2label = {0: "negative", 1: "positive"}
label2id = {"negative": 0, "positive": 1}

In [26]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [27]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id, dropout=0.2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
training_args = TrainingArguments(
    output_dir="imdbreviews_classification_distilbert_v02",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/home/camilo/.local/share/virtualenvs/MachineLearning-OGHLGSeN/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
preds_output = trainer.predict(tokenized_test)
preds_output.metrics